In [ ]:
import pandas as pd
from scipy.sparse import hstack
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

final_users_courses = pd.read_csv('/content/drive/My Drive/BT4222_Group_3_Submission/cleaned_datasets/final_users_courses.csv')


Mounted at /content/drive


### Textual Preprocessing

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

additional_stopwords = ['learn', 'course', 'students', 'introduction', 'understand', 'overview',
    'concepts', 'designed', 'including', 'using', 'used', 'provides',
    'students', 'aimed', 'teach', 'taught', 'include', 'includes', 'including',
    'understanding', 'module', 'modules', 'cover', 'covers', 'covered',
    'introduce', 'introduces', 'introduced', 'overview', 'knowledge', 'skills',
    'learning', 'lecture', 'lectures', 'topic', 'topics', 'area', 'areas',
    'focus', 'focuses', 'focused', 'approach', 'approaches', 'principles',
    'principle', 'content', 'contents', 'key', 'features', 'feature',
    'element', 'elements', 'basis', 'basic', 'basics', 'foundation', 'foundations',
    'beginner', 'intermediate', 'advanced', 'level', 'levels', 'intended',
    'objective', 'objectives', 'goal', 'goals', 'outcome', 'outcomes', 'aim', 'aims',
    'gain', 'gains', 'apply', 'effective', 'efficient', 'specialize', 'specialization',
    'specialise', 'certificate', 'certification', 'you', 'will', 'need',
    'want', 'have', 'able', 'well', 'one', 'two', 'three', 'first', 'second',
    'third', 'success', 'successful', 'sucessfully', 'like', 'many' , 'much', 'also', 'use', 'uses', 'used',
    'work', 'works', 'working', 'workshop', 'workshops', 'provide', 'provides', 'provided']

def process_text(text):
    processed_text = ""
    if isinstance(text, str):
        text = text.lower()
    else:
        text = ''
    tokens = word_tokenize(text)
    alphabetic_tokens = [token for token in tokens if token.isalpha()]
    stop_words = stopwords.words('english') + additional_stopwords
    important_words = ['no', 'not', 'never', 'none', 'neither','otherwise', 'but', 'however', 'though', 'very',
                       'extremely', 'exceptionally', 'definitely', 'too']
    filtered_tokens = [token for token in alphabetic_tokens if token not in stop_words or token in important_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    processed_text = " ".join(lemmatized_words)

    return processed_text

final_users_courses['processed_review'] = final_users_courses['Review'].apply(process_text)
final_users_courses['processed_skill'] = final_users_courses['Course Skills'].apply(process_text)
final_users_courses['processed_institution'] = final_users_courses['Institution'].apply(process_text)
final_users_courses['processed_description'] = final_users_courses['Description'].apply(process_text)

print("data_df:")
print(final_users_courses[['processed_review', 'processed_skill', 'processed_institution']].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


data_df:
                                    processed_review  \
0  very friendly complete easy follow highly reco...   
1  budgeting scheduling project give learner prac...   
2                              very detailed helpful   
3  initiating planning project give learner pract...   
4                                            xie xie   

                                     processed_skill  \
0    nutrition organic food food safety food science   
1  schedule project management budget budget esti...   
2  negotiation analysis negotiation developing po...   
3  project management project planning project pl...   
4      grammar communication chinese language speech   

          processed_institution  
0           stanford university  
1  university california irvine  
2           university michigan  
3  university california irvine  
4             peking university  


In [ ]:
final_users_courses

,Review,Reviewer,Individual Rating,Course Name,Institution,Overall Ratings,Level,Duration,Num of Reviews,Course Skills,Description,Date,Demeaned Rating,Popularity,processed_review,processed_skill,processed_institution,processed_description
0,this is a very friendly course. it is complete...,lorenia a,5,child nutrition and cooking,stanford university,4.8,Beginner level,11,9,"nutrition, organic food, food safety, food sci...",\n Welcome to the 'ChatGPT Playground for Beg...,2015-08-07,0.000000,43.2,very friendly complete easy follow highly reco...,nutrition organic food food safety food science,stanford university,welcome playground beginner world ai guided pr...
1,doing this course budgeting and scheduling pro...,vijayakumar m n,5,budgeting and scheduling projects,"university of california, irvine",4.7,NaN,6,8,", schedule, project management, budget, budget...",\n Define milestones and create a milestone s...,2015-08-07,0.000000,37.6,budgeting scheduling project give learner prac...,schedule project management budget budget esti...,university california irvine,define milestone create milestone scheduleiden...
2,very detailed and helpful!,swati s,5,successful negotiation: essential strategies a...,university of michigan,4.8,Beginner level,17,18,"negotiation analysis, negotiation, developing ...",Understand the negotiation strategies and skil...,2015-08-07,0.173913,86.4,very detailed helpful,negotiation analysis negotiation developing po...,university michigan,negotiation strategy necessary negotiation ana...
3,doing this course initiating and planning proj...,vijayakumar m n,5,initiating and planning projects,"university of california, irvine",4.7,NaN,7,22,", project management, project, planning, proje...",\n Identify project stakeholdersDefine the ro...,2015-08-07,0.000000,103.4,initiating planning project give learner pract...,project management project planning project pl...,university california irvine,identify project stakeholdersdefine role respo...
4,xie xie,camila g,5,chinese for beginners,peking university,4.8,NaN,26,18,"grammar, communication, chinese language, spee...",\n Take your Excel productivity to the next l...,2015-08-08,0.250000,86.4,xie xie,grammar communication chinese language speech,peking university,take excel productivity next leverage capabili...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134137,super informative!,dana r,5,the bits and bytes of computer networking,google,4.7,Beginner level,27,47,"domain name system dns, ipv4, network model, t...",This course is designed to provide a full over...,2020-10-10,0.166667,220.9,super informative,domain name system dns network model troublesh...,google,full computer networking everything fundamenta...
134138,this course is great. it is really hands on an...,william h,5,technical support fundamentals,google,4.8,Beginner level,28,147,"binary code, customer support, linux, troubles...",This course is the first of a series that aims...,2020-10-10,0.846154,705.6,great really hand great job trying keep motiva...,binary code customer support linux troubleshoo...,google,series prepare role support specialist world i...
134139,best thing i did for myself this year.,caroline s,5,the science of well-being,yale university,4.9,NaN,19,37,"gratitude, happiness, meditation, savoring ,",\n NEW TEEN VERSION AVAILABLE HERE: https://w...,2020-10-10,0.000000,181.3,best thing year,gratitude happiness meditation savoring,yale university,new teen version available http engage series ...
134140,tons of great information! im absolutely amaze...,thomas v,5,the bits and bytes of computer networking,google,4.7,Beginner level,27,47,"domain name system dns, ipv4, network model, t...",This course is designed to provide a full over...,2020-10-10,0.777778,220.9,ton great information im absolutely amazed amo...,domain name system dns network model troublesh...,google,full computer networking everything fundamenta...


### User Profile and Course Profile

In [ ]:
user_profile = final_users_courses.groupby('Reviewer').agg({
    'Course Name': lambda x:', '.join(x.unique()),
    'processed_skill': lambda x:', '.join(x.unique()),
    'Duration':'mean',
    'Overall Ratings':'mean',
    'Popularity':'mean',
    'Num of Reviews' : 'mean',
    'processed_institution':  lambda x:', '.join(x.unique()),
    'processed_description':  lambda x:', '.join(x.unique()),
}).reset_index()

user_profile.columns = ['Reviewer', 'Courses', 'Skills', 'Average Duration', 'Average Rating', 'Average Popularity', 'Average Reviews', 'Institution', 'Description']

user_profile



,Reviewer,Courses,Skills,Average Duration,Average Rating,Average Popularity,Average Reviews,Institution,Description
0,- a u s,"neural networks and deep learning, improving d...",artificial neural network backpropagation pyth...,20.500000,4.850000,316.550000,65.000000,,give participant broad study core infrastructu...
1,.,"neural networks and deep learning, sequence mo...",artificial neural network backpropagation pyth...,20.500000,4.850000,316.550000,65.000000,,give participant broad study core infrastructu...
2,01fe18bec055,"what is data science?, data science methodolog...",data science big data machine deep data mining...,13.666667,4.633333,143.300000,30.666667,"ibm,",define data science importance today various p...
3,031_ankit r,programming for everybody getting started with...,python syntax semantics programming language c...,12.666667,4.733333,483.766667,101.333333,"university michigan, ibm",everyone programming computer python construct...
4,17g113 v a v,"supply chain principles, learn to program: the...","procurement supply chain transporting, python ...",21.000000,4.733333,89.233333,18.333333,"georgia institute technology, university toron...",solid supply chain supply chain leverage graph...
...,...,...,...,...,...,...,...,...,...
22570,zuwei l,"excel skills for business: essentials, excel/v...","graph spreadsheet microsoft excel chart, micro...",24.333333,4.866667,109.300000,22.333333,"macquarie university, university colorado boulder",fundamental machine various aws service necess...
22571,zuzana s,"fundamentals of graphic design, initiating and...","creativity graphic design theory color theory,...",18.250000,4.800000,80.050000,16.750000,"california institute art, university californi...",implement fundamental color visual rhythm patt...
22572,zvinodashe m,programming for everybody getting started with...,python syntax semantics programming language c...,16.800000,4.760000,365.260000,75.800000,"university michigan, pwc, imperial college london",everyone programming computer python construct...
22573,zvonimir k,"what is data science?, tools for data science,...",data science big data machine deep data mining...,12.500000,4.625000,150.925000,32.500000,ibm,define data science importance today various p...


In [ ]:
course_profile = final_users_courses.groupby('Course Name').agg({
    'Reviewer': lambda x: ', '.join(x.unique()),
    'processed_skill': lambda x: ', '.join(x.unique()),
    'Duration': 'mean',
    'processed_institution': lambda x: ', '.join(x.unique()),
    'processed_description': lambda x: ', '.join(x.unique()),
    'Overall Ratings': 'mean',
    'Num of Reviews': 'mean',
    'Popularity' :  'mean',
}).reset_index()

course_profile.columns = [
    'Course Name', 'Reviewers', 'Skills', 'Average Duration',
    'Institution', 'Description', 'Average Rating', 'Average Reviews', 'Average Popularity'
]

course_profile

,Course Name,Reviewers,Skills,Average Duration,Institution,Description,Average Rating,Average Reviews,Average Popularity
0,a crash course in causality: inferring causal ...,"hao l, chang l, arnab s, michael l, mark f, vi...",instrumental variable propensity score matchin...,18.0,university pennsylvania,heard phrase correlation not equal equal causa...,4.7,525.0,2467.5
1,accounting data analytics with python,"chris a, minjeong k, scott u, sanjay k, n b",data preparation python programming linear reg...,42.0,university illinois,know operate software help create run python p...,4.3,84.0,361.2
2,addiction treatment: clinical skills for healt...,"shelby s, kim h, michelle m, anna k, kevin, ni...",substance disorder substance disorder screenin...,14.0,yale university,singular improve care patient substance disord...,4.8,1.0,4.8
3,agile meets design thinking,"saeed s s, sachin s, rajan a, andrew p, karen ...",software development product management agile ...,9.0,university virginia,evaluate explain relevance practice agile deve...,4.7,3.0,14.1
4,agile with atlassian jira,"beth p, chris g, rakesh, john b, sagar s, asho...",project management agile management scrum soft...,12.0,atlassian,discus common foundational practice agile meth...,4.7,8.0,37.6
...,...,...,...,...,...,...,...,...,...
234,wind energy,"anand d, pablo m p, chaitanya k, vishnu s, dan...",wind energy material wind power wind energy en...,34.0,technical university denmark dtu,welcome wind energy give aspect wind energy en...,4.7,7.0,32.9
235,write a feature length screenplay for film or ...,"kannan v, ashna g, manuel a r s, david m, prac...",creative writing filmmaking script writing fil...,93.0,michigan state university,write complete film screenplay finished script...,4.6,196.0,901.6
236,write your first novel,"ian f, bianca, r r, mary h, ayan c, danial s",creative writing story telling novel writing c...,127.0,michigan state university,write complete novel word finished manuscript ...,4.3,297.0,1277.1
237,writing and editing: word choice and word order,"raghavendra s, anthony k, samuel h, abigail b,...",creativity time management persuasion writing ...,20.0,university michigan,word choose change decision people make syntax...,4.7,1.0,4.7


### Text Vectorization through TF-IDF and Numerical Normalization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

#initialize vectorizers for each text field
tfidf_vectorizer_skills = TfidfVectorizer(stop_words='english')
tfidf_vectorizer_institution = TfidfVectorizer(stop_words='english')
tfidf_vectorizer_description = TfidfVectorizer(stop_words='english')

#vectorize Skills
combined_skills = pd.concat([user_profile['Skills'], course_profile['Skills']])
skills_tfidf = tfidf_vectorizer_skills.fit_transform(combined_skills)
user_skills_tfidf = skills_tfidf[:user_profile.shape[0]]
course_skills_tfidf = skills_tfidf[user_profile.shape[0]:]

#vectorize Institutions
combined_institutions = pd.concat([user_profile['Institution'], course_profile['Institution']])
institution_tfidf = tfidf_vectorizer_institution.fit_transform(combined_institutions)
user_institution_tfidf = institution_tfidf[:user_profile.shape[0]]
course_institution_tfidf = institution_tfidf[user_profile.shape[0]:]

#vectorize Descriptions
combined_descriptions = pd.concat([user_profile['Description'], course_profile['Description']])
description_tfidf = tfidf_vectorizer_description.fit_transform(combined_descriptions)
user_description_tfidf = description_tfidf[:user_profile.shape[0]]
course_description_tfidf = description_tfidf[user_profile.shape[0]:]

In [ ]:
#numeric values, calculate normalized
def prepare_vectors_and_calculate_similarity(user_values, course_values):
    combined_values = np.concatenate([user_values, course_values])

    #handle NaN
    imputer = SimpleImputer(strategy='median')
    imputed_values = imputer.fit_transform(combined_values.reshape(-1, 1)).flatten()

    scaler = MinMaxScaler()
    scaled_values = scaler.fit_transform(imputed_values.reshape(-1, 1)).flatten()

    #scaled values back into user and course
    normalized_user_values = scaled_values[:len(user_values)]
    normalized_course_values = scaled_values[len(user_values):]

    normalized_user_values = normalized_user_values.reshape(-1, 1)
    normalized_course_values = normalized_course_values.reshape(-1, 1)

    similarity = cosine_similarity(normalized_user_values, normalized_course_values)
    return similarity

duration_similarity = prepare_vectors_and_calculate_similarity(user_profile['Average Duration'].values, course_profile['Average Duration'].values)
rating_similarity = prepare_vectors_and_calculate_similarity(user_profile['Average Rating'].values, course_profile['Average Rating'].values)
reviews_similarity = prepare_vectors_and_calculate_similarity(user_profile['Average Reviews'].values, course_profile['Average Reviews'].values)
popularity_similarity = prepare_vectors_and_calculate_similarity(user_profile['Average Popularity'].values, course_profile['Average Popularity'].values)


### Content Based Filtering - Weights added to various features


In [ ]:
#weights for each attribute
weight_skills = 1/7
weight_institution = 1/7
weight_description = 1/7
weight_duration = 1/7
weight_rating = 1/7
weight_popularity = 1/7
weight_reviews = 1/7

#cosine similarity for each vectorized text field
similarity_skills = cosine_similarity(user_skills_tfidf, course_skills_tfidf)
similarity_institution = cosine_similarity(user_institution_tfidf, course_institution_tfidf)
similarity_description = cosine_similarity(user_description_tfidf, course_description_tfidf)

#apply weights to each attribute's similarity score
weighted_skills_similarity = similarity_skills * weight_skills
weighted_institution_similarity = similarity_institution * weight_institution
weighted_description_similarity = similarity_description * weight_description
weighted_duration_similarity = duration_similarity * weight_duration
weighted_rating_similarity = rating_similarity * weight_rating
weighted_reviews_similarity = reviews_similarity * weight_reviews
weighted_popularity_similarity = popularity_similarity * weight_popularity

#update combined similarity calculation to include weighted duration similarity
combined_similarity = (weighted_skills_similarity + weighted_institution_similarity +
                       weighted_description_similarity + weighted_duration_similarity + weighted_popularity_similarity
                       + weighted_reviews_similarity + weighted_rating_similarity)

#convert the updated combined similarity to a DataFrame
user_course_interaction_matrix = pd.DataFrame(combined_similarity,
                                             index=user_profile['Reviewer'],
                                             columns=course_profile['Course Name'])

user_course_interaction_matrix
print("Updated User-Course Interaction Matrix:")
print(user_course_interaction_matrix)

Updated User-Course Interaction Matrix:
Course Name   a crash course in causality: inferring causal effects from observational data  \
Reviewer                                                                                      
- a u s                                                0.573516                               
.                                                      0.573516                               
01fe18bec055                                           0.583958                               
031_ankit r                                            0.595012                               
17g113 v a v                                           0.586661                               
...                                                         ...                               
zuwei l                                                0.597285                               
zuzana s                                               0.596414                               
zvinodashe

In [ ]:
top_10_courses = user_course_interaction_matrix.apply(lambda x: x.nlargest(10).index.tolist(), axis=1)

print("Top 10 Recommended Courses for Each User:")
print(top_10_courses)

Top 10 Recommended Courses for Each User:
Reviewer
- a u s         [sequence models, improving deep neural networ...
.               [sequence models, improving deep neural networ...
01fe18bec055    [data science methodology, what is data scienc...
031_ankit r     [introduction to artificial intelligence ai, w...
17g113 v a v    [supply chain principles, learn to program: th...
                                      ...                        
zuwei l         [excel skills for business: intermediate i, ex...
zuzana s        [indigenous canada, fundamentals of graphic de...
zvinodashe m    [data-driven decision making, mathematics for ...
zvonimir k      [data analysis with python, data science metho...
zw              [introduction to data science in python, pytho...
Length: 22575, dtype: object


### Sample Recommendation

In [ ]:
if 'zw' in user_course_interaction_matrix.index:
    # Get the courses with the highest similarity scores for user 'zw'
    top_recommendations = user_course_interaction_matrix.loc['zw'].sort_values(ascending=False)

    # Print the top 5 recommended courses
    print("Top 5 Recommendations for User 'zw':")
    print(top_recommendations.head(5))
else:
    print("No data available for user 'zw'")


Top 5 Recommendations for User 'zw':
Course Name
introduction to data science in python                   0.852244
python data structures                                   0.775278
applied machine learning in python                       0.771507
programming for everybody getting started with python    0.759376
using python to access web data                          0.751875
Name: zw, dtype: float64


### Evaluation using train and eval test sets to determine optimal weights




### Train Set

In [ ]:
train = pd.read_csv('/content/drive/My Drive/BT4222_Group_3_Submission/cleaned_datasets/train.csv')
import pandas as pd

null_counts = train.isnull().sum()
print(null_counts)


Review                 112
Reviewer                 0
Individual Rating        0
Course Name              0
Institution              0
Overall Ratings        115
Level                16006
Duration                 0
Num of Reviews           0
Course Skills            0
Description              0
Date                     0
Demeaned Rating          0
Popularity             115
dtype: int64


### Evaluation Set

In [ ]:
eval = pd.read_csv('/content/drive/My Drive/BT4222_Group_3_Submission/cleaned_datasets/eval.csv')
actual_courses = eval.groupby('Reviewer')['Course Name'].agg(list)
def evaluate_recommendations(recommendations, actual):
    precision_list = []
    recall_list = []

    # Iterate over each reviewer's recommendations and actual courses
    for reviewer, recs in recommendations.items():
        acts = actual.get(reviewer, [])
        true_positives = set(recs) & set(acts)

        precision = len(true_positives) / len(recs) if recs else 0
        recall = len(true_positives) / len(acts) if acts else 0

        precision_list.append(precision)
        recall_list.append(recall)

    average_precision = sum(precision_list) / len(precision_list) if precision_list else 0
    average_recall = sum(recall_list) / len(recall_list) if recall_list else 0
    f1 = 2 * (average_precision * average_recall) / (average_precision + average_recall)

    return average_precision, average_recall, f1

### Weight Tuning

In [ ]:
additional_stopwords = ['learn', 'course', 'students', 'introduction', 'understand', 'overview',
    'concepts', 'designed', 'including', 'using', 'used', 'provides',
    'students', 'aimed', 'teach', 'taught', 'include', 'includes', 'including',
    'understanding', 'module', 'modules', 'cover', 'covers', 'covered',
    'introduce', 'introduces', 'introduced', 'overview', 'knowledge', 'skills',
    'learning', 'lecture', 'lectures', 'topic', 'topics', 'area', 'areas',
    'focus', 'focuses', 'focused', 'approach', 'approaches', 'principles',
    'principle', 'content', 'contents', 'key', 'features', 'feature',
    'element', 'elements', 'basis', 'basic', 'basics', 'foundation', 'foundations',
    'beginner', 'intermediate', 'advanced', 'level', 'levels', 'intended',
    'objective', 'objectives', 'goal', 'goals', 'outcome', 'outcomes', 'aim', 'aims',
    'gain', 'gains', 'apply', 'effective', 'efficient', 'specialize', 'specialization',
    'specialise', 'certificate', 'certification', 'you', 'will', 'need',
    'want', 'have', 'able', 'well', 'one', 'two', 'three', 'first', 'second',
    'third', 'success', 'successful', 'sucessfully', 'like', 'many' , 'much', 'also', 'use', 'uses', 'used',
    'work', 'works', 'working', 'workshop', 'workshops', 'provide', 'provides', 'provided']

def process_text(text):
    processed_text = ""
    if isinstance(text, str):
        text = text.lower()
    else:
        text = ''
    tokens = word_tokenize(text)
    alphabetic_tokens = [token for token in tokens if token.isalpha()]
    stop_words = stopwords.words('english') + additional_stopwords
    important_words = ['no', 'not', 'never', 'none', 'neither','otherwise', 'but', 'however', 'though', 'very',
                       'extremely', 'exceptionally', 'definitely', 'too']
    filtered_tokens = [token for token in alphabetic_tokens if token not in stop_words or token in important_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    processed_text = " ".join(lemmatized_words)

    return processed_text
train['processed_review'] = train['Review'].apply(process_text)
train['processed_skill'] = train['Course Skills'].apply(process_text)
train['processed_institution'] = train['Institution'].apply(process_text)
train['processed_description'] = train['Description'].apply(process_text)

In [ ]:
user_profile = train.groupby('Reviewer').agg({
    'Course Name': lambda x:', '.join(x.unique()),
    'processed_skill': lambda x:', '.join(x.unique()),
    'Duration':'mean',
    'Overall Ratings':'mean',
    'Popularity':'mean',
    'Num of Reviews' : 'mean',
    'processed_institution':  lambda x:', '.join(x.unique()),
    'processed_description':  lambda x:', '.join(x.unique()),
}).reset_index()

user_profile.columns = ['Reviewer', 'Courses', 'Skills', 'Average Duration', 'Average Rating', 'Average Popularity', 'Average Reviews', 'Institution', 'Description']

course_profile = train.groupby('Course Name').agg({
    'Reviewer': lambda x: ', '.join(x.unique()),
    'processed_skill': lambda x: ', '.join(x.unique()),
    'Duration': 'mean',
    'processed_institution': lambda x: ', '.join(x.unique()),
    'processed_description': lambda x: ', '.join(x.unique()),
    'Overall Ratings': 'mean',
    'Num of Reviews': 'mean',
    'Popularity' :  'mean',
}).reset_index()

course_profile.columns = [
    'Course Name', 'Reviewers', 'Skills', 'Average Duration',
    'Institution', 'Description', 'Average Rating', 'Average Reviews', 'Average Popularity'
]

#initialize vectorizers for each text field
tfidf_vectorizer_skills = TfidfVectorizer(stop_words='english')
tfidf_vectorizer_institution = TfidfVectorizer(stop_words='english')
tfidf_vectorizer_description = TfidfVectorizer(stop_words='english')

#vectorize Skills
combined_skills = pd.concat([user_profile['Skills'], course_profile['Skills']])
skills_tfidf = tfidf_vectorizer_skills.fit_transform(combined_skills)
user_skills_tfidf = skills_tfidf[:user_profile.shape[0]]
course_skills_tfidf = skills_tfidf[user_profile.shape[0]:]

#vectorize Institutions
combined_institutions = pd.concat([user_profile['Institution'], course_profile['Institution']])
institution_tfidf = tfidf_vectorizer_institution.fit_transform(combined_institutions)
user_institution_tfidf = institution_tfidf[:user_profile.shape[0]]
course_institution_tfidf = institution_tfidf[user_profile.shape[0]:]

#vectorize Descriptions
combined_descriptions = pd.concat([user_profile['Description'], course_profile['Description']])
description_tfidf = tfidf_vectorizer_description.fit_transform(combined_descriptions)
user_description_tfidf = description_tfidf[:user_profile.shape[0]]
course_description_tfidf = description_tfidf[user_profile.shape[0]:]

#numeric values, calculate normalized
def prepare_vectors_and_calculate_similarity(user_values, course_values):
    combined_values = np.concatenate([user_values, course_values])

    imputer = SimpleImputer(strategy='median')
    imputed_values = imputer.fit_transform(combined_values.reshape(-1, 1)).flatten()

    scaler = MinMaxScaler()
    scaled_values = scaler.fit_transform(imputed_values.reshape(-1, 1)).flatten()

    normalized_user_values = scaled_values[:len(user_values)]
    normalized_course_values = scaled_values[len(user_values):]

    normalized_user_values = normalized_user_values.reshape(-1, 1)
    normalized_course_values = normalized_course_values.reshape(-1, 1)

    similarity = cosine_similarity(normalized_user_values, normalized_course_values)
    return similarity

duration_similarity = prepare_vectors_and_calculate_similarity(user_profile['Average Duration'].values, course_profile['Average Duration'].values)
rating_similarity = prepare_vectors_and_calculate_similarity(user_profile['Average Rating'].values, course_profile['Average Rating'].values)
reviews_similarity = prepare_vectors_and_calculate_similarity(user_profile['Average Reviews'].values, course_profile['Average Reviews'].values)
popularity_similarity = prepare_vectors_and_calculate_similarity(user_profile['Average Popularity'].values, course_profile['Average Popularity'].values)

similarity_skills = cosine_similarity(user_skills_tfidf, course_skills_tfidf)
similarity_institution = cosine_similarity(user_institution_tfidf, course_institution_tfidf)
similarity_description = cosine_similarity(user_description_tfidf, course_description_tfidf)

#change the weight configurations for testing
weight_configurations = [
    {'Skills': 1/2, 'Institution': 1/8, 'Description': 1/8, 'Duration': 1/16, 'Rating': 1/16, 'Popularity': 1/16, 'Reviews': 1/16},
    {'Skills': 3/4, 'Institution': 1/16, 'Description': 1/16, 'Duration': 1/32, 'Rating': 1/32, 'Popularity': 1/32, 'Reviews': 1/32},
    {'Skills': 1/7, 'Institution': 1/7, 'Description': 1/7, 'Duration': 1/7, 'Rating': 1/7, 'Popularity': 1/7, 'Reviews': 1/7},
]

for i, weights in enumerate(weight_configurations, start=1):
    weighted_skills_similarity = similarity_skills * weights['Skills']
    weighted_institution_similarity = similarity_institution * weights['Institution']
    weighted_description_similarity = similarity_description * weights['Description']
    weighted_duration_similarity = duration_similarity * weights['Duration']
    weighted_rating_similarity = rating_similarity * weights['Rating']
    weighted_popularity_similarity = popularity_similarity * weights['Popularity']
    weighted_reviews_similarity = reviews_similarity * weights['Reviews']

    combined_similarity = (weighted_skills_similarity + weighted_institution_similarity +
                           weighted_description_similarity + weighted_duration_similarity +
                           weighted_popularity_similarity + weighted_reviews_similarity +
                           weighted_rating_similarity)

    user_course_interaction_matrix = pd.DataFrame(combined_similarity,
                                                  index=user_profile['Reviewer'],
                                                  columns=course_profile['Course Name'])



    top_10_courses = user_course_interaction_matrix.apply(lambda x: x.nlargest(10).index.tolist(), axis=1)
    precision, recall, F1 = evaluate_recommendations(top_10_courses, actual_courses)
    print(f"Precision: {precision}, Recall: {recall}, F1: {F1}")
    print("Top 10 Recommended Courses for Each User:")
    print(top_10_courses)
    print("\n")


Precision: 0.028239202657810547, Recall: 0.2583283705271193, F1: 0.05091285888700074
Top 10 Recommended Courses for Each User:
Reviewer
- a u s         [neural networks and deep learning, improving ...
.               [sequence models, neural networks and deep lea...
01fe18bec055    [what is data science?, data science methodolo...
031_ankit r     [programming for everybody getting started wit...
17g113 v a v    [supply chain principles, supply chain logisti...
                                      ...                        
zuwei l         [excel skills for business: essentials, excel ...
zuzana s        [fundamentals of graphic design, initiating an...
zvinodashe m    [data-driven decision making, python data stru...
zvonimir k      [tools for data science, what is data science?...
zw              [neural networks and deep learning, introducti...
Length: 22575, dtype: object


Precision: 0.02575415282392297, Recall: 0.2334612122943176, F1: 0.04639073565059345
Top 10 Recommended Cour

### Final Weights: 1/7 to all Features


Test Set

In [ ]:
test_df = pd.read_csv('/content/drive/My Drive/BT4222_Group_3_Submission/cleaned_datasets/test.csv')
user_course_interaction_matrix = user_course_interaction_matrix.astype(float)
top_n_recs = user_course_interaction_matrix.apply(lambda x: list(x.nlargest(10).index), axis=1)
actual_courses = test_df.groupby('Reviewer')['Course Name'].agg(list)

def evaluate_recommendations(recommendations, actual):
    precision_list = []
    recall_list = []

    # Iterate over each reviewer's recommendations and actual courses
    for reviewer, recs in recommendations.items():
        acts = actual.get(reviewer, [])
        true_positives = set(recs) & set(acts)

        precision = len(true_positives) / len(recs) if recs else 0
        recall = len(true_positives) / len(acts) if acts else 0

        precision_list.append(precision)
        recall_list.append(recall)

    average_precision = sum(precision_list) / len(precision_list) if precision_list else 0
    average_recall = sum(recall_list) / len(recall_list) if recall_list else 0
    f1 = 2 * (average_precision * average_recall) / (average_precision + average_recall)

    return average_precision, average_recall, f1

precision, recall, f1 = evaluate_recommendations(top_n_recs, actual_courses)
f1_score = 2 * (precision * recall) / (precision + recall)

# Print the results
print(f"Precision: {round(precision, 4)}, Recall: {round(recall, 4)}, F1 Score: {round(f1_score, 4)}")

Precision: 0.022, Recall: 0.1988, F1 Score: 0.0396
